There are two whole numbers:<br>
1 < a,b <100<br>
<br>
One scientist("Sum") get provided with sum of numbers,<br>
another  ("Prod") get provided with product of numbers.<br>
Both scientists know that numbers 1 < a,b <100.<br>
<br>
Determine the numbers being based on the following dialog: <br>
    Prod: I don't know the numbers;<br>
    Sum: I know it;<br>
    Prod: then I know the numbers; <br>
    Sum: then I know the numbers too.<br>

In [13]:
# unique_prods = both numbers prime

# Prod: I don't know the numbers; >>> at least one number is not prime

# Sum: I know it; >>> cannot be a sum of prime numbers

# Prod: then I know the numbers; >>> if divided on factors there is only one option
#                                    of not being a sum of prime numbers

# Sum: then I know the numbers too. >>> if divided on summands there is only one option 
#                                       that leaves Prod with only one option on previous step


# The above looks quite complicated and layered, and it seems that there will be a lot of different lists
# to be managed in process, e.g. lists for all possible products, sums, sets of factors, prime numbers.
# But thinking of a way to make the solution more readable and efficient, using numpy matrices comes to mind.
# If we create one matrix for products and one for sums, where the indices correspond to the numbers in range(2,100),
# then we can conveniently exclude the options from both matrices by indices or values, step by step.
# I assume that when we complete all the steps, there will be only one pair of indices left in both matrices.
# And that combination of numbers will be the answer.

# Some additional notes:
# - The products and sums matrices are symmetric, 
#   so it's enough use the upper triangles only.
# - Deleting an option by value from sums matrix means
#   deleting the whole opposite diagonal with that value for sums



# !!!!!!!!!the below is to be deleted
# long story short:
# for prod, if divided on factors there is only one option of not being a sum of prime numbers

# so there should exist only one product (not only one)
# which doesn't consist of primes only,
# factors of which can only be split in one way
# in order to form a sum which can only be formed by non prime numbers

# and then there is only one sum that provides only one option for product

In [14]:
import numpy as np
import itertools

def list_primes(upto):
    odds = np.arange(3, upto + 1, 2)
    isprime = np.ones((upto - 1) // 2, dtype=bool)
    for factor in odds[:int(upto**.5)//2]:
        if isprime[(factor-2)//2]:
            isprime[(factor*3-2)//2::factor] = 0
    return np.insert(odds[isprime], 0, 2)

primes = np.array(list_primes(99))
numbers = np.arange(2, 100)

products = np.zeros((100, 100), dtype=int)
products[2:, 2:] = np.triu(numbers[:, None] * numbers[None, :])

products_original = products.copy()

sums = np.zeros((100, 100), dtype=int)
sums[2:, 2:] = np.triu(numbers[:, None] + numbers[None, :])

print(products, '\n')
print(sums)

[[   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    4 ...  194  196  198]
 ...
 [   0    0    0 ... 9409 9506 9603]
 [   0    0    0 ...    0 9604 9702]
 [   0    0    0 ...    0    0 9801]] 

[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   4 ...  99 100 101]
 ...
 [  0   0   0 ... 194 195 196]
 [  0   0   0 ...   0 196 197]
 [  0   0   0 ...   0   0 198]]


In [15]:
def remove_diagonal(x, y, *, mark=0):
    '''Removes the whole opposite diagonal in upper triangle from products and sums matrices'''
    rng = np.arange(max(x + y - 99, 0), min(x + y + 1, 100))
    upper_triangle_diagonal = rng[:(len(rng)+1)//2], rng[:-(len(rng)+2)//2:-1]

    sums[upper_triangle_diagonal] = mark
    products[upper_triangle_diagonal] = mark

def remove_by_index(x, y):
    '''Removes the value in upper triangle from products and sums matrices'''
    sums[x, y] = 0
    products[x, y] = 0

def remove_by_prod(prod):
    pass

def remove_by_sum(sum):
    pass

def available_values():
    '''Returns indices of available values for the next step'''
    # return np.nonzero(products)
    return np.positive(sums)

def number_available():
    '''Returns the number of available values for the next step'''
    return np.sum(sums > 0)

'Total available pairs of numbers: ', number_available()

('Total available pairs of numbers: ', 4851)

In [16]:
# Prod: I don't know the numbers; >>> at least one number is not prime
# Step 1. Exclude all values where both numbers are prime.

# Sum: I know it; >>> cannot be a sum of prime numbers
# Step 2. Exclude all values that can be formed by sum of prime numbers
# i.e exclude opposite diagonal for each where value from Step 1.

prime_combos = itertools.combinations_with_replacement(primes, 2)

for combo in prime_combos:
    remove_diagonal(*combo, mark=-1) # marking prime sums diagonals for step 3

'Available pairs of numbers after Step 1 and Step 2: ', number_available()

('Available pairs of numbers after Step 1 and Step 2: ', 1932)

In [17]:
# Prod: then I know the numbers; >>> if divided on factors there is only one option 
#                                    of not being a sum of prime numbers
# Step 3. For each product find indices of equal values, count pairs which make sums
#         that lay in a diagonal of a prime number sum. Than filter out single counts
#         but only when there are multiple sets of factors (else prod could have guessed earlier).
# TEMPORARILY PROD TABLE WILL NOT BE MODIFIED AT STEP 2 TO SAFE PRODS IN PRIME DIAGONALS
unique_prods = np.unique(products)[2:]
print(unique_prods.shape)
good_prods = np.array([])
for prod in unique_prods:
    corresponding_sums = sums[np.where(products_original == prod)]
    if len(corresponding_sums) > 1 and np.sum(corresponding_sums != -1) == 1:
        # print('keep this prod', prod, corresponding_sums)
        good_prods = np.append(good_prods, prod)
        products[np.where(products_original == prod)] = -2
    # if this gives array of len > 1 and there is only one -1,
    # then this is the value we need to keep
len(good_prods)

(1445,)


375

In [19]:
# Sum: then I know the numbers too. >>> if divided on summands there is only one option 
#                                       that leaves Prod with only one option on previous step
# Step 4. Divide each sum on summands, count combos that multiply into one of prods values
#         from Step 3. Then filter out single counts. There sould be only one such. 

unique_sums = np.unique(sums)[2:]
print(unique_sums.shape)
good_sums = np.array([])
for sum in unique_sums:
    corresponding_prods = products[np.where(sums == sum)]
    if np.sum(corresponding_prods == -2) == 1:
        good_sums = np.append(good_sums, sum)
        # print('keep this sum', sum, corresponding_prods, np.where(sums == sum))
len(good_sums)

(81,)


7